In [ ]:
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

### 2. EDA (E xploratory Data Analysis) 란
- EDA : 탐색적 데이터 분석
    - 데이터를 탐색하고 이해하기 위한 과정으로, 데이터 세트의 구조, 패턴, 상관관계, 이상치 등을 파악하는 것을 목표
    - 방법 : 요약 통계, 시각화, 상관 분석, 분포 분석, 클러스터링 등

In [ ]:
# 데이터프레임 정보 확인
print(train.info())

# 데이터프레임 기술 통계량 확인
print(train.describe())

### 4. 일별 따릉이 대여량(number_of_rentals) 데이터 시각화

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

fe = fm.FontEntry(fname = 'NotoSansKR-Regular.otf', name = 'NotoSansKR')
fm.fontManager.ttflist.insert(0, fe)
plt.rc('font', family='NotoSansKR')

In [ ]:
plt.figure(dpi=150) # 그림 해상도 150
x = train['number_of_rentals'].index
y = train['number_of_rentals'].values

plt.title('따릉이 대여량')
plt.xlabel('일별')
plt.xticks(rotation=45)
plt.ylabel('일별 따릉이 대여량')
plt.plot(x, y)
plt.show()

### 5. 일별 따릉이 대여량(number_of_rentals) 이상치 시각화
- 위의 그래프에 일별 따릉이 대여량 값 중 8000을 이상치를 판단하는 기준이라고 가정하고, plt.hlines() 함수를 사용하여 y 값이 8000인 수평선을 그려보자.

In [ ]:
plt.figure(dpi=150)
x = train['number_of_rentals'].index
y = train['number_of_rentals'].values

plt.title('따릉이 대여량')
plt.xlabel('일별')
plt.xticks(rotation=45)
plt.ylabel('일별 따릉이 대여량')
# xmin=0과 xmax=len(x)는 x축 범위 설정, 즉 x의 처음과 끝을 의미
plt.hlines(y=8000, xmin=0, xmax=len(x), color='red', linestyles='dotted')
plt.plot(x, y)
plt.show()

### 6. train 데이터프레임(DataFrame)에서 이상치 값 상위 5행(row) 데이터 추출
- 대여량이 8000을 넘지 않는 이상치를 확인

In [ ]:
train[train['number_of_rentals']<=8000].head()

### 7. train 데이터프레임(DataFrame)에서 이상치 평균값 추출

In [ ]:
train[train['number_of_rentals']<=8000].mean()

### 8.train 데이터프레임(Dataframe)에서 대여량 이상치에 따른 피처(feature)별 시각화
- 강수 형태, 강수 확률에서 8000 이상과 이하의 값 사이에 큰 차이가 있다는 것을 확인할 수 있다.

In [ ]:
low = train[train['number_of_rentals']<=8000].mean()
high = train[train['number_of_rentals']>8000].mean()

def compare(idx):
    x = ['8000<= '+low.index[idx], '8000> '+low.index[idx]]
    y = [low.values[idx], high.values[idx]]

    plt.figure(figsize=(8,5))
    plt.title(low.index[idx]+" 의 평균")
    plt.xlabel('피처')
    plt.ylabel('수치')
    plt.bar(x, y)
    plt.show()

for i in range(8):
    compare(i)

### 9.일별 강수 형태(precipitation_form) 변화를 나타내는 선형 그래프
- plot(선 그래프)을 이용하여 시간 흐름에 따른 강수 형태를 그래프로 시각화
- 그래프의 모양, 변동 패턴 등을 살펴봄으로써 강수 형태 데이터의 특성 파악

[결론]
- 시간의 흐름에 따른 강수 형태를 봤을 때, 뚜렷한 패턴은 보이지 않는다.
- 그러나 값이 0인 데이터, 즉 비가 오지 않은 날이 많아 보인다.

In [ ]:
plt.figure(figsize=(30,6))

a = plt.plot(train['precipitation_form'], color='g')

plt.ylabel('precipitation_form')
plt.xlabel('Days')
plt.show()

### 10.연도별 따릉이 대여량(number_of_rentals) 평균 시각화
- date_time 변수를 활용하여 연도별 강수 형태의 변동을 살펴봄

[결론]
- 연도별로 따릉이 대여량이 꾸준히 증가하다.

In [ ]:
train['year'] = train['date_time'].apply(lambda x : x.split('-')[0])

train.groupby('year').mean()[['number_of_rentals']].plot(figsize=(15,5))
plt.show()

### 13.파생변수 생성 : 연도(year) : EDA를 통해 얻은 인사이트를 이용해서 결론을 도출해보자.
- 강수 변수 : 따릉이 대여량 큰 차이
    - 정확한 분석을 위하여 date_time 변수 EDA
- date_time 변수 : 연도, 월별
    - 뚜렷한 증사 추세
- date_time 변수 : 일별
    - 뚜렷한 증감세가 없다가, 월말에 급상승

### 2. train 데이터프레임에서 data_time 열(column) 제거 및 열 확인
문자형 변수는 텍스트 형태의 데이터로 구성되어 있기 때문에, 회귀분석에서 직접 사용하기 어렵다. 이를 사용하기 위해서는 문자형 변수를 수치형으로 변환해야 하는 과정이 필요하다.

따라서, 이번 프로젝트에서는 문자형 변수를 제거(drop)하고 수치형 데이터만 사용할 것.

In [ ]:
train = train.drop(['date_time'], axis=1)
print(train.columns)

### 3. test 데이터프레임에서 date_time 열(column) 제거 및 열 확인

In [ ]:
test = test.drop(['date_time'], axis=1)

### 4. Feature Selection
- X 변수에는 train 데이터프레임에서 column_names 리스트에 해당하는 열들을 선택하여 핟당
- y 변수에는 train 데이터프레임에서 number_of_rentals 열 할당

### 5. LinearRegression 모델 정의

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

### 6. LinearRegression 모델 학습

In [ ]:
model.fit(X, y)

### 7. LinearRegression 모델 예측

In [ ]:
predict = model.predict(X, y)
print(predict[:10])

### 8. 예측된 따릉이 대여량을 submission 파일에 추가하기

In [ ]:
submission['number_of_rentals'] = predict
submission.head()

### 9. 데이터프레임(DataFrame)을 CSV 파일로 저장하기

In [ ]:
submission.to_csv('submission_csv', index=False)